In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from langchain.agents.middleware import wrap_model_call,ModelRequest,ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model=init_chat_model("gpt-5-mini")
small_model=init_chat_model("gpt-5-nano")

@wrap_model_call
def state_based_model(request:ModelRequest,handler:Callable[[ModelRequest],ModelResponse]):
    """Select model based on conversation length"""
    #request.messages is a shortcut for request.state['messages']
    message_count=len(request.messages)

    if message_count>10:
        model=large_model
    else:
        model=small_model
    
    request=request.override(model=model)

    return handler(request)

In [12]:
from langchain.agents import create_agent

agent=create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt='You are a roleplaying a real life helpful office intern'
)

In [13]:
from langchain.messages import HumanMessage
response=agent.invoke({"messages":[HumanMessage(content="Did you water the office plant today?")]})
print(response["messages"][-1].content)

I can’t physically water it from here, so I haven’t done it today. If you want, I can water it now with your go-ahead, or set up a reminder and log it in the plant care sheet for future days. I can also check the soil moisture first if you’d like. What would you prefer?


In [14]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [15]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

Given how it’s looking now (new leaves and only slight yellowing at the edges), we probably don’t need to repot immediately. General guidance:

- Typical timing: most houseplants need repotting every 12–24 months; fast growers may need it yearly, slow growers every 2–3 years.  
- Signs it needs repotting sooner:
  - Roots coming out of drainage holes or circling the surface when you lift it out.  
  - Water runs straight through the pot or spills out the sides quickly.  
  - Soil compacts and dries out very fast, or the plant shows stunted growth despite good care.  
  - Pot is cracked or too small for the plant visually.

- Best practice: repot in spring (start of active growth), using fresh potting mix and a pot 1–2 inches larger in diameter for small plants (2–4 inches for larger ones). Make sure it has drainage holes.

If you like, I can check the root situation tomorrow and, if needed, repot it then. Which would you prefer?


In [16]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-mini-2025-08-07
